In [1]:
from numpy.linalg import inv
import numpy as np
from math import *
from copy import deepcopy
from sympy import *

In [2]:
x,y,z = symbols('x y z')
init_printing(use_unicode=True)
tolerance = 0.0001
g = 9.806
k = 0.00341
testFunction1 = log(cosh(x*sqrt(g*k))) - 50
testFunction2 = 4*cos(x)- exp(2*x)


## Método da bisseção

Esse método precisa de dois valores iniciais, a e b. Esses valores devem ser um positivo e outro negativo. Ainda, a raiz deve estar contida no intervalo [a,b].

O processo seguido pelo método é definir o ponto candidato a raiz como o meio do intervalo [a,b] e testar se a função nesse ponto candidato está dentro do intervalo de tolerância. Se não estiver, ele muda o valor do intervalo analisado de acordo com o seguinte critério:


- Se a função aplicada no ponto definido como candidato a raiz deu um valor maior que zero, substitui o limite superior do intervalo analisado pelo próprio ponto candidato a raiz;
- Se a função aplicada no ponto definido como candidato a raiz deu um valor menor que zero, substitui o limite inferior do intervalo analisado pelo próprio ponto candidato a raiz.

In [3]:
def bissectionMethod(function,a,b,tol):
	"""
	Implements: Bissection Method
	Arguments:
		function: function to apply the method (sympy function)
		a: begin of the analyzed interval (number)
		b: end of the analyzed interval (number)
		tol: error tolerance (number)
	Return: root of analyzed function (number) 
	"""
	iterations = 0
	while(abs(a-b) > tol and iterations < 1000):
		rootPoint = (a+b)/2
		rootValue = function.subs(x,rootPoint)
		if(rootValue > 0):
			b = rootPoint 
		else:
			a = rootPoint 
		iterations+= 1
	return rootPoint

In [4]:
print("Function 1: ",bissectionMethod(testFunction2,-30.0,10.0,tolerance))
print("Function 2: ",bissectionMethod(testFunction1,-1000.0,1000.0,tolerance))

Function 1:  -7.8539276123046875
Function 2:  277.22102403640747



## Método de Newton e Método da Secante

No método de Newton expandimos a série de Taylor, truncamos no termo linear e igualamos a zero. Dessa forma, conseguimos encontrar a raiz de uma função qualquer dado apenas um ponto inicial e a função analisada.

O processo seguido por essa função é o de usar o quociente entre f(x) e f`(x) para estimar uma espécie de ¨passo¨ h que aproxima o valor x do valor correto da raiz. Caso a função seja linear apenas uma iteração é necessária. 


> A diferença entre o método de Newton e o método da secante é que no método de Newton calculamos a derivada da maneira tradicional, ao passo que no método da secante calculamos a derivada pela própria definição de derivada.

In [7]:
def newtonMetod(function,initialValue,tol):
	"""
	Implements: Newthon Method
	Arguments:
		function: function to apply the method (sympy function)
		initialValue: starting point (number)
		tol: error tolerance (number)
	Return: root of analyzed function (number) 
	"""
	iterations = 100
	rootPoint = initialValue
	for i in range (iterations):
		lastRoot = rootPoint  
		functionDerivative =  diff(function,x)
		rootPoint  = rootPoint - function.evalf(subs={x:rootPoint})/functionDerivative.evalf(subs={x:rootPoint})
	if(abs(rootPoint - lastRoot) < tol):
		return rootPoint
	else:
		return "convergence not reached"

In [8]:
print("Function 1 with initial point 200: ",newtonMetod(testFunction1,200,tolerance))
print("Function 1 with initial point -200: ",newtonMetod(testFunction1,-200,tolerance))
print("Function 2 with initial point -10: ",newtonMetod(testFunction2,-10,tolerance))
print("Function 2 with initial point -15: ",newtonMetod(testFunction2,-15,tolerance))
print("Function 2 with initial point -8: ",newtonMetod(testFunction2,-8,tolerance))


Function 1 with initial point 200:  277.220996556061
Function 1 with initial point -200:  -277.220996556061
Function 2 with initial point -10:  -10.9955742876346
Function 2 with initial point -15:  -14.1371669411539
Function 2 with initial point -8:  -7.85398159629905


In [22]:
def secantMethod(function,initialValue,tol):
	"""
	Implements: Secant Method
	Arguments:
		function: function to apply the method (sympy function)
		initialValue: starting point (number)
		tol: error tolerance (number)
	Return: root of analyzed function (number) 
	"""
	iterations = 1000
	rootPoint = initialValue
	delta = 0.001
	lastRoot = rootPoint
	rootPoint = lastRoot + delta
	fa = function.evalf(subs={x:lastRoot})
	for i in range (iterations):
		fi = function.evalf(subs={x:rootPoint})
		# We use temp as a X of the last Iteration
		temp = lastRoot
		# lastRoot assumes rot values before rootPoint update it value
		lastRoot = rootPoint
		# The line below is the main difference to Newton Method. We calculate the derivative using the definition
		rootPoint = rootPoint - (fi * (rootPoint-temp))/(fi-fa)
		if(abs(rootPoint - lastRoot) < tol):
			return rootPoint
		else:
			fa = fi
	return "convergence not reached"

In [24]:
print("Function 1 with initial point 200: ",secantMethod(testFunction1,200,tolerance))
print("Function 1 with initial point -200: ",secantMethod(testFunction1,-200,tolerance))
print("Function 2 with initial point -10: ",secantMethod(testFunction2,-10,tolerance))
print("Function 2 with initial point -15: ",secantMethod(testFunction2,-15,tolerance))
print("Function 2 with initial point -8: ",secantMethod(testFunction2,-8,tolerance))

Function 1 with initial point 200:  277.220996556061
Function 1 with initial point -200:  -277.220996556061
Function 2 with initial point -10:  -10.9955742876351
Function 2 with initial point -15:  -14.1371669411539
Function 2 with initial point -8:  -7.85398159629840



## Método da Interpolação Inversa

Nesse método construímos um polinômio quadrático usando Interpolação de Langrange de x em função de y e calculando seu valor para y = 0.

O processo seguindo por essa função é calcular os interpoladores de Lagrange e encontrar um valor de `x*`. Se esse x* não estiver dentro da tolerância definida,  o valor do maior dos pontos x utilizados na interpolação (esses x começam por pontos definidos pelo usuário) é substituído pelo x* encontrado e o processo é repetido.

In [36]:
def inverseInterpolationMethod(function,x1,x2,x3,tol):
	"""
	Implements: Inverse Interpolation Method
	Arguments:
		function: function to apply the method (sympy function)
		x1,x2,x3: points to use in Langrange`s interpolation (number)
		tol: error tolerance (number)
	Return: root of analyzed function (number) 
	"""
	# We start x* (root of last iteration) as 10^36
	lastRoot = pow(10,36)
	iterations = 1000
	list_x = [x1,x2,x3]
	list_y = [0,0,0]
	#print(function.subs(x,1))
	for i in range (iterations):
		list_y[0] = function.evalf(subs={x:list_x[0]})
		list_y[1] = function.evalf(subs={x:list_x[1]})
		list_y[2] = function.evalf(subs={x:list_x[2]})
		rootPoint = (list_y[1]*list_y[2]*list_x[0])/((list_y[0]-list_y[1])*(list_y[0]-list_y[2])) + (list_y[0]*list_y[2]*list_x[1])/((list_y[1]-list_y[0])*(list_y[1]-list_y[2])) + (list_y[0]*list_y[1]*list_x[2])/((list_y[2]-list_y[0])*(list_y[2]-list_y[1]))
		if(abs(rootPoint-lastRoot) < tol):
			return rootPoint
		else:
			i = list_y.index(max(list_y))
			list_x[i] = rootPoint
			list_y[i] = function.subs(x,list_x[i])
			lastRoot = rootPoint
	return "Convergence not reached"

In [40]:
print("Function 1 with initial points (200,250,260): ",inverseInterpolationMethod(testFunction1,200,250,260,tolerance))
print("Function 1 with initial points (-200,-250,-260): ",inverseInterpolationMethod(testFunction1,-200,-250,-260,tolerance))
print("Function 2 with initial points (-13,-12,-8):: ",inverseInterpolationMethod(testFunction2,-13,-12,-8,tolerance))
print("Function 2 with initial points (-16,-14,-18): ",inverseInterpolationMethod(testFunction2,-16,-14,-18,tolerance))
print("Function 2 with initial points (-5,-7,-9): ",inverseInterpolationMethod(testFunction2,-6,-7,-9,tolerance))

Function 1 with initial points (200,250,260):  277.220996556061
Function 1 with initial points (-200,-250,-260):  -277.220996556061
Function 2 with initial points (-13,-12,-8)::  -7.85398158146460
Function 2 with initial points (-16,-14,-18):  -14.1371659644094
Function 2 with initial points (-5,-7,-9):  -7.10621448704260


### Métodos auxiliares

Antes de iniciar o estudo de problemas multidimensionais, vamos definir os seguintes métodos auxiliares:
 

In [42]:
def jacobian(funcArray):
	global symbolsList
	functionArray = deepcopy(funcArray)
	jacobian = []
	for i in range(functionArray.size):
		temp = []
		for j in range(len(symbolsList)):
			temp.append(diff(functionArray[i], symbolsList[j]))
		jacobian.append(temp)
	return jacobian


def changeValuesMatrix(matrix, valueArray):
	global symbolsList
	functionMatrix = deepcopy(matrix)
	for i in range(len(functionMatrix)):
		for j in range(len(functionMatrix[i])):
			for k in range(len(symbolsList)):
				functionMatrix[i][j] = functionMatrix[i][j].subs(symbolsList[k], valueArray[k])
	return functionMatrix


def changeValuesArray(array, valueArray):
	global symbolsList
	functionArray = deepcopy(array)
	for i in range(len(functionArray)):
		for k in range(len(symbolsList)):
				functionArray[i] = functionArray[i].subs(symbolsList[k], valueArray[k])
	return functionArray


def changeValuesArrayBroyden(array, valueArray):
	global symbolsList
	functionArray = deepcopy(array)
	for i in range(len(functionArray)):
		for j in range(len(functionArray[i])):
			for k in range(len(symbolsList)):
				functionArray[i][j] = functionArray[i][j].subs(symbolsList[k], valueArray[k])
	return functionArray

## Método de Newton (caso multidimensional)

Expandimos a série de Taylor até a primeira ordem e somos capazes de obter a seguinte relação:
$$f(x) = -J \Delta   X$$, onde J é a nossa matriz jacobiana. Sendo assim, usamos a seguinte relação para resolver o sistema:

$$X^{k+1} = X^{k} + \Delta X^{k}= X^{k}-J(X^{k})^{-1}f(X^{k})$$ 


In [ ]:
def multiDimensionalNewtonMethod(functionArray, X0, tol, symbols):
	"""
    Inserir detalhes da funcao 
	"""
	iterations = 15000
	jacob = jacobian(functionArray)
	lastX = X0

	# define symbols that will be used
	global symbolsList
	symbolsList = symbols

	for i in range(iterations):
		j = changeValuesMatrix(jacob, lastX)
		f = changeValuesArray(functionArray, lastX)

		j_np = np.array(j).astype(np.float64)
		f_np = np.array(f).astype(np.float64)
		
		deltaX = -np.dot(inv(j_np),f_np)
		lastX = lastX + deltaX

		tolk = np.linalg.norm(deltaX, ord=2) / np.linalg.norm(lastX, ord=2)
		if (tolk < tol):
			return lastX

	return "Convergence not reached"

functionArray1 = np.array([16*(x**4)+16*(y**4)+(z**4)-16, (x**2)+(y**2)+(z**2)-3, (x**3)-y+z-1])
functionArray2 = np.array([x+2*y-2,(x**2)+4*(y**2)-4])

In [ ]:
##Inserir casos de teste aqui


## Método de Broyden

É similar ao método de Newton, porém a matriz Jacobiana não é calculada numericamente em cada iteração. Esse método usa uma matriz jacobiana aproximada B, dada por:

$$B^{k+1} = B^{k} + \frac{(Y-B^{k}\Delta X)(\Delta X)^{T})}{(\Delta X)^{T}(\Delta X)}$$


In [43]:
def multiDimensionalBroydenMethod(functionArray, X0, B0, tol, symbols):
    """
    Inserir detalhes da funcao 
	"""
	iterations = 15000
	X_list = [X0]
	B_list = [B0] 		# receive the jacobian of start

	# define symbols that will be used
	global symbolsList
	symbolsList = symbols

	for i in range(1, iterations+1):
		j_np = B_list[i-1]

		f_ant = changeValuesArrayBroyden(functionArray, X_list[i-1])

		f_np_ant = np.array(f_ant).astype(np.float64)
		
		deltaX = -np.dot(inv(j_np),f_np_ant)

		X_list.append(np.add(X_list[i-1], deltaX.transpose())[0])

		f = changeValuesArrayBroyden(functionArray, X_list[i])
		f_np = np.array(f).astype(np.float64)

		lastY = f_np - f_np_ant

		#print("### iteraction: ",i," B: ",j_np, " f_ant: ",f_ant, " f_np_ant: ",f_np_ant, " xlist -1: ", X_list[i-1], "deltaX: ",deltaX)

		tolk = np.linalg.norm(deltaX, ord=2) / np.linalg.norm(X_list[i], ord=2)
		if (tolk < tol):
			return X_list[i]
		else:
			deltaX_transp = deltaX.transpose()

			#print("### iteraction: ",i," deltaX_transp",deltaX_transp)

		b_dividendo = np.dot(lastY-np.dot(np.asmatrix(B_list[i-1]), deltaX), deltaX_transp)
		b_divisor = np.dot(deltaX_transp, deltaX)

		next_B = B_list[i-1] + np.divide(b_dividendo, b_divisor)

		#print("### nextB: ", next_B.tolist())

		B_list.append(next_B.tolist())

		print("### iteraction: ",i," ---- VALUE",X_list[i])

	return "Convergence not reached"

In [ ]:
## Inserir exemplos aqui